In [32]:
def duration(link):
    from bs4 import BeautifulSoup
    import requests
    import urllib.request
    import pandas as pd
    import re
    import warnings
    warnings.filterwarnings('ignore')
    
    response=requests.get(link)
    html_contents = response.text
    html_soup = BeautifulSoup(html_contents,'html.parser')
    gde=[]
    kogda=[]
    b1=html_soup.find('div',id='ExperienceHolder').find_all('b')
    m=html_soup.find('div',id='ExperienceHolder').find_all('em')
    for i in b1:
        gde.append(i.text.strip())
    for k in m:
        kogda.append(k.text.strip())
    gde=[x for x in gde if "Достиж" not in x]
    gde=[x for x in gde if "Рекоменд" not in x]
    gde=[x for x in gde if "улучш" not in x]
    gde=[x for x in gde if "Результ" not in x]
    gde=[x for x in gde if "Обязанн" not in x]
    gde=[x for x in gde if "достиж" not in x]
    gde=gde[:min(len(gde),len(kogda))]
    kogda=kogda[:min(len(gde),len(kogda))]
    d = {'должность':gde,'период':kogda}
    uh=pd.DataFrame(d)
    uh['предприятие']=uh['должность'].shift(-1)
    uh['отрасль']=uh['период'].shift(-1)
    
    uh['отрасль']=uh['отрасль'].str.extract('\\t(\w+)').fillna(uh['отрасль'])
    
    uhh=uh.iloc[::2, :]
    regex=re.compile('(\(\d+.лет|\(\d+.год)')
    uhh['год']=uhh['период'].str.extract(regex)
    uhh['год']=uhh['год'].str.extract(r'(\d+)').fillna(0)
    uhh['мес']=uhh['период'].str.extract('(\d+).мес').fillna(0)
    uhh['мес1']=uhh['мес'].astype(int)/12
    uhh['duration']=uhh['год'].astype(int)+uhh['мес1'].round(2)
    uhh['link_1']=link
    uhh=uhh[['link_1','предприятие','должность','период','отрасль','duration']].reset_index(drop=True)
    return uhh